# Python for (open) Neuroscience

_Lecture 3.0_ - PsychoPy

Matteo De Matola

Luigi Petrucco

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/lectures/Lecture3.0_PsychoPy.ipynb)

## [PsychoPy](https://psychopy.org/) is a Python library to program psychological experiments.  

![PsychoPy logo](https://psychopy.org/_static/Psychopy%20Logotext.png)

Like many Python libraries, PsychoPy is:
- Free
- Open source
- The result of a large-scale collaborative effort

If you use PsychoPy in published work, remember to cite the authors:

[Peirce, J., Gray, J. R., Simpson, S., MacAskill, M., Höchenberger, R., Sogo, H., ... & Lindeløv, J. K. (2019). PsychoPy2: Experiments in behavior made easy. Behavior Research Methods, 51, 195-203](https://link.springer.com/article/10.3758/s13428-018-01193-y)

The more we reward open-source efforts, the more we free research from market rules.

On to the lecture!

PsychoPy contains functions and classes to:
- Generate visual stimuli

- Generate auditory stimuli

- Collect responses from subjects 

- Time events

- Interact with external hardware, including:
    - Keyboards
    - Response boxes
    - Cameras
    - Eye tracking devices
    - Neuroimaging systems

Most psychological / neuroscientific research involves a person watching things on a screen and pressing buttons during neuroimaging.

That's also what my research involves (at least, in part). 

Therefore, this lecture will focus on:
- Generating and displaying visual stimuli
- Interacting with keyboards and EEG systems

However, all concepts can be transferred to other kinds of stimuli and research settings.

Try to think:
- "Ok, that's how PsychoPy represents a stimulus"
- "Ok, that's how PsychoPy loops over experimental trials"

Instead of:
- "Aw, that's just how you generate a _visual_ stimulus"
- "Aw, that's just how you loop over experimental trials in an _EEG_ experiment"

Most neuroscientific research involves a person watching things on a screen and pressing buttons during neuroimaging.

Therefore, we need code to:

- Generate visual stimuli (the things)

- Represent the screen and interact with it

- Present stimuli with the right timing

- Represent different experimental conditions
    - Their balance
    - Their order

- Represent buttons and encode buttonpresses

- Represent the neuroimaging system and interact with it

### Step 1: Generate visual stimuli

- The `visual` module

Contains classes and functions to:

- Draw a variety of pre-set stimuli 
    - `ShapeStim`
        - `Rect`
        - `Circle`
        - `Polygon`
        - `Line`
        - `Pie`

- Display images and draw patterns
    - `SimpleImageStim`
    - `GratingStim`
    - etc.

Go [here](https://psychopy.org/api/visual/index.html) for the full list of options and the associated documentation.

### Step 2: Represent the screen and interact with it

Stimuli are drawn on a specific _window_. This window can be as large as the screen, or smaller.  
- The `visual` module
    - The `Window` class

Opening a window is as simple as:

In [ ]:
from psychopy import visual

window = visual.Window()

`Window` objects represent the canvas where you draw your stimuli.   

But that canvas cannot exist without some hardware to support it. That hardware is your monitor (or _screen_).

Different monitors have different physical properties:
- Width
- Pixel resolution
- ...

... And can be used in different ways:
- Distance from screen
    - Vital if your stimuli are in [degrees of visual angle (°VA)](https://en.wikipedia.org/wiki/Visual_angle)

Specify the physical properties of your monitor, or the `Window` will be initialised with default settings that might be suboptimal for you:
- The `monitors` module
    - The `Monitor` class

In [ ]:
from psychopy import monitors

monitor = monitors.Monitor(name="matteos_monitor")
monitor.setSizePix([1366,768])       
monitor.setWidth(47)                 
monitor.setDistance(65)               
monitor.saveMon()                    

### Step 3: Present stimuli with the right timing

Timing can be very important. You need to make sure that:

- Stimuli appear when you want them to

- Stimuli remain on for as long as you want them to

- Stimuli are synchronised with neural events of interest
    - For example: if you are doing EEG, you usually need **very** fast transitions (`ms` or sub-`ms`)

Each stimulus must have a given display time (measured in seconds).

For example:

In [ ]:
display_time = 0.1 # one tenth of a second = 100 ms

This display time must be transformed into a number of frames.

A frame is one of many still images that compose a complete moving picture ([Wikipedia](https://en.wikipedia.org/wiki/Film_frame)). 

Different screens can display different numbers of frames in one second
- Frame (or _refresh_) rate (Hz) ([Wikipedia](https://en.wikipedia.org/wiki/Frame_rate))
    - Often used interchangeably, but _refresh rate_ is better when talking about screens (and _frame rate_ when talking about GPUs) 

To transform a display time into a number of frames, you must multiply it by your screen's refresh rate:

In [ ]:
refresh_rate = 60 # Hz
frames_per_stimulus = display_time*refresh_rate

The number of frames per item will serve as a range to iterate over:

In [ ]:
for frame in range(frames_per_stimulus):
    # display item

But, how do you actually _display_ an item?

- The `flip()` method of `Window` objects
    - Move between frames 

- The `setAutoDraw()` method of all `ShapeStim` objects
    - Draw the stimulus on every frame, without explicit frame-by-frame instruction

In [ ]:
window = visual.Window(color="black")
stimulus = visual.rect.Rect(fillColor="white")

stimulus.setAutoDraw(True)
for frame in range(frames_per_stimulus):
    window.flip()

Practicals 3.0.0:

- Open an Anaconda prompt or terminal
- Type `conda activate course-env`
- Type `pip install psychopy`
- In any code editor, create a `.py` file where you:
    - Initialise a `Monitor` with a given name, then set its pixel size
    - Initialise a `Window` object as follows:
        - `window = visual.Window(size=monitor.getSizePix(), fullscr=False, monitor=monitor.name, color="black")`
    - Create a stimulus of your choice 
    - Display the stimulus for 10 seconds at a 60 Hz refresh rate
    - Save the `.py` file and run it
        - From either the code editor or terminal

### Step 4: Represent different experimental conditions

In neuroscientific experiments, we usually study how a subject responds to some stimulation (sensory, emotional, magnetic, electrical, etc.).

For statistical power, we want to measure the same response several times. 

Therefore, we want an experiment with multiple _trials_: stereotyped, repeated presentations of the same stimulus or stimulus sequence.

In psychological experiments, we also want to study how a subject responds to different _kinds_ of stimulation (e.g., seeing a square vs. seeing a circle).

Therefore, we want an experiment with multiple _conditions_: variations of the stimulus or stimulus sequence presented to the subject within a given trial.

We need code to:
- Define different conditions
- Administer each condition for multiple trials
    - The number of trials per condition will vary depending on our design: balance
- Administer the conditions in the order that we want 
    - That's often random

**Define different conditions, 1: the `conditions.csv` file**

In PsychoPy, you need to define your conditions in an external `.csv` file 

- One row per condition

- One column per condition parameter:
    - Presence vs. absence of a given stimulus (1,0)
    - Colour of the stimulus ("yellow", "blue",...)
    - Whatever you can think of...

- If you want to give higher weight to one condition (i.e., present it more times than others), you can put it in multiple rows 

**Define different conditions, 2: the `data.importConditions` function**

Each experimental trial corresponds to one condition. Therefore, each experimental trial is one row of the `conditions.csv` file.

- PsychoPy implements trials as iterations over the rows of the `conditions.csv` file

- To do so, it reads the `conditions.csv` file and transforms it in an [iterable](https://realpython.com/python-iterators-iterables/#getting-to-know-python-iterables) (i.e., an object that you can iterate over)

- This is done by the `importConditions` function from module `data` 

In [ ]:
from psychopy import data

my_conditions = data.importConditions("conditions.csv")

**Administer each condition for multiple trials, in the order you want: the `data.TrialHandler` class**

The output of `data.importConditions` is meant to be the input to `data.TrialHandler`

The class `TrialHandler` from module `data` implements methods to:
- Iterate over conditions (as read by `data.importConditions`)
- Do it as many times as you want (one trial per condition vs. multiple trials per condition)
- Do it in the order you want (sequential vs. random)

In [ ]:
my_trials = data.TrialHandler(trialList=my_conditions, 
                              nReps=1,
                              method="random")
for trial in my_trials:
    # do things

Pay attention to the `method` argument:

- You probably want to pass `random` to avoid order effects:

    shuffles the conditions, and _"all conditions occur once before the second repeat"_

- You can also pass `fullRandom`, but be aware that 

    _"you could potentially run all trials of one condition before any trial of another"_

- Finally, you can pass `sequential` if that's OK for your design. 

    _"presents the conditions in the order they appear in the list"_ (i.e., in the `conditions.csv` file)

### Step 5: Represent buttons and encode buttonpresses

- The `event` module
    - The `getKeys()` function

The `event` module contains classes and functions to _"handle input from keyboard, mouse and joystick"_

The `getKeys()` function continuously checks for keypresses and continuously returns a list of the presses that it finds.
- If you press `left`, `getKeys()` will detect it and return a list of type: `["left"]`

You can specify the keys that you are interested in with the `keyList` argument (`type: list`). 
- If you do so, `getKeys()` will check exclusively for the keys contained in `keyList`

In [ ]:
from psychopy import event

pressed_key = event.getKeys(keyList=["left","right"]) 

Once you have stored the keypress in a variable, you can use it for response scoring:

In [ ]:
if pressed_key[0] == "left":
    response = 1        # means "correct"
elif pressed_key[0] == "right":
    response = 0        # means "wrong"

Often, response timing is important. 
- Example: reaction time is a dependent variable

PsychoPy provides a wealth of tools to measure time. 

These tools are scattered across the `core` and `clock` modules
- I believe this might change in the future, with all timing tools migrating to `clock`
- In the meantime, you can visit [`core`](https://www.psychopy.org/api/core.html)'s and [`clock`](https://www.psychopy.org/api/clock.html)'s documentation to choose the tool that suits your needs

A simple case study: measuring reaction times

- The `Clock` class from the `core` module
    - The `getTime()` method
    - The `reset` method

Pseudocode:

In [ ]:
from psychopy import core

response_clock = core.Clock()
for trial in my_trials:
    # present your stimuli 
    if response:
        reaction_time = response_clock.getTime()
        response_clock.reset

(_pseudocode_ is a code sketch that illustrates a concept without necessarily following the syntax of any given language)

Resetting the clock after taking the time is **VERY** important. 

If you forget to do it, you will end up taking incremental times:

- First reaction time: `clock.getTime()` - $t_{start}$

    where $t_{start}$ is the time when you initialised your clock (i.e., when `response_clock = core.Clock()` was executed)

- Second reaction time: `clock_getTime()` + first reaction time

- Third reaction time: `clock_getTime()` + second reaction time

- etc...

What you really want is: 
- Any reaction time: `clock.getTime()` - $t_{start}$

    where $t_{start}$ is the time when you presented you stimulus (i.e., the beginning of a frame or trial)

Real code:

In [ ]:
from psychopy import visual, core, event

window = visual.Window(color="black")
stimulus = visual.rect.Rect(fillColor="white")

response_clock = core.Clock()

for trial in my_trials:
    stimulus.setAutoDraw(True)
    window.callOnFlip(response_clock.reset) # analogous to setAutoDraw
    event.clearEvents()                     # clears old  keypresses 
    for frame in range(frames_per_stimulus):
        keys = event.getKeys(keyList=["left","right"])
        if len(keys) > 0:
            response = keys[0]   
            reaction_time = response_clock.getTime() 
            break
        window.flip()

Practicals 3.0.1:

Modify the script from the previous practical. You want to:
- Accept responses 
- Take reaction times
- Print the reaction time

When you are done with your modifications, run the script to see if it works.

### Step 6: Represent the neuroimaging system and interact with it

This step will vary depending on the neuroimaging technique and your hardware set-up. 

But, what are we even talking about?

When you do a behavioural experiment during neuroimaging, you want a link between neuroimaging and behavioural data: 

- Mark EEG time series at stimulus presentation times

- Present stimuli when an MRI scanner is ready for them, i.e., at the right moment of an acquisition sequence
    - Disclaimer: I have no MRI expertise

**Problem:** 

The PsychoPy code for the behavioural task runs in one computer

The neuroimaging data are acquired by another / the MRI scans are controlled by another

You need a piece of hardware to connect two computers and carry signals between them. 

This is often a parallel port, but it can well be a serial port.

Simplifying:
- A parallel port is a hardware interface that can transmit multiple signals in parallel. It is connected to a cable with multiple threads (akin to a nervous fibre) ([Wikipedia](https://en.wikipedia.org/wiki/Parallel_port))
- A serial port is a hardware interface that can transmit one signal at a time ([Wikipedia](https://en.wikipedia.org/wiki/Serial_port))

PsychoPy has a `hardware` module that contains:

- The `parallel` submodule, to send signals through parallel ports
- The `serial` submodule, to send signals through serial ports

I'll make the example with `parallel`, but `serial` is similar in concept. In both cases, you need to:
- Import the appropriate submodule
- Initialise an object that represents your port. This object will have one key attribute: the port's _address_ (basically, its name)
- Write code to have signals travelling from / to the port

Example: marking an EEG trace through a parallel port

In [ ]:
from psychopy import parallel

port = parallel.ParallelPort(address="0x0378") # can vary
port.setPin(pinNumber=1, 
            state=1)
# display your stimulus
port.setPin(pinNumber=1,
            state=0)

Go [here](https://psychopy.org/hardware/fMRI.html) for a tutorial on synchronising stimulus presentation with the activity of MRI scanners. 

**Final considerations**

PsychoPy is a great piece of software:
- You can run virtually any type of visual / auditory experiment
- You can interact with quite a lot of hardware (though I only have EEG expertise, eheh)
- All you need is a bit of Python expertise

However, it has flaws:
- The library has an associated GUI called Builder. It is meant to:
    - Empower people who can't code
    - Accelerate prototyping for the people who can code
- PsychoPy developers promote the Builder heavily, so most online tutorials will be Builder-centred
    - Bad news for coders
- In general, the documentation for the codebase is:
    - Hard to find, as it's buried under a myriad articles about the GUI
    - Poorly maintained 
    - But these are my personal opinions   